In [39]:
#!pip install datasets
#!pip install transformers
#!git clone https://github.com/besacier/ALFFA_PUBLIC.git
#!pip install wandb

In [2]:
from pathlib import Path
import os
import warnings
import random

import re
import json
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict, load_dataset
import librosa
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Audio

from IPython.display import display, HTML


import torch
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
# chargement des données d'entrainement
# récupération des ids et des transcriptions à partir du fichier text
with open("/content/ALFFA_PUBLIC/ASR/FONGBE/data/train/text") as file_name:
    data_train = file_name.read().split("\n")

ids = []
textes = []

for line in data_train:
    parties = line.split(' ', 1)
    if len(parties) == 2:
        identifiant, texte = parties
        ids.append(identifiant)
        textes.append(texte)
    else:
        ids.append(parties[0])
        textes.append('')

df1 = pd.DataFrame({'Identifiant': ids, 'Texte': textes})

# récupération des ids et des chemines audio à partir du fichier wav.scp
with open("/content/ALFFA_PUBLIC/ASR/FONGBE/data/train/wav.scp") as file_name:
    data = file_name.read().split("\n")

ids = []
liens = []

for line in data:
    parties = line.split(' ', 1)
    if len(parties) == 2:
        identifiant, lien = parties
        ids.append(identifiant)
        liens.append(lien)
    else:
        ids.append(parties[0])
        liens.append('')

df2 = pd.DataFrame({'Identifiant': ids, 'Liens': liens})

# Remplacer "./data" par "/content/ALFFA_PUBLIC/ASR/FONGBE/data" dans la colonne "Liens"
df2['Liens'] = df2['Liens'].str.replace('./data', '/content/ALFFA_PUBLIC/ASR/FONGBE/data').str.strip()

# Fusionnez les deux DataFrames sur la colonne "Identifiant"
df_train = df2.merge(df1, on='Identifiant', how='inner')

df_train = df_train.drop(['Identifiant'], axis=1)
df_train = df_train.rename(columns={'Texte': 'transcription'})
df_train = df_train.rename(columns={'Liens': 'file'})
df_train = df_train.replace('', pd.NA).dropna()

<ipython-input-4-6c777c4972f5>:41: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Liens'] = df2['Liens'].str.replace('./data', '/content/ALFFA_PUBLIC/ASR/FONGBE/data').str.strip()


In [5]:
# chargement des données test
# récupération des ids et des transcriptions à partir du fichier text
with open("/content/ALFFA_PUBLIC/ASR/FONGBE/data/test/text") as file_name:
    data_test = file_name.read().split("\n")

ids = []
textes = []

for line in data_test:
    parties = line.split(' ', 1)
    if len(parties) == 2:
        identifiant, texte = parties
        ids.append(identifiant)
        textes.append(texte)
    else:
        ids.append(parties[0])
        textes.append('')

df1 = pd.DataFrame({'Identifiant': ids, 'Texte': textes})

# récupération des ids et des chemines audio à partir du fichier wav.scp
with open("/content/ALFFA_PUBLIC/ASR/FONGBE/data/test/wav.scp") as file_name:
    data = file_name.read().split("\n")

ids = []
liens = []

for line in data:
    parties = line.split(' ', 1)
    if len(parties) == 2:
        identifiant, lien = parties
        ids.append(identifiant)
        liens.append(lien)
    else:
        ids.append(parties[0])
        liens.append('')

df2 = pd.DataFrame({'Identifiant': ids, 'Liens': liens})

# Remplacer "./data" par "/content/ALFFA_PUBLIC/ASR/FONGBE/data" dans la colonne "Liens"
df2['Liens'] = df2['Liens'].str.replace('./data', '/content/ALFFA_PUBLIC/ASR/FONGBE/data').str.strip()

# Fusionnez les deux DataFrames sur la colonne "Identifiant"
df_test = df2.merge(df1, on='Identifiant', how='inner')

df_test = df_test.drop(['Identifiant'], axis=1)
df_test = df_test.rename(columns={'Texte': 'transcription'})
df_test = df_test.rename(columns={'Liens': 'file'})
df_test = df_test.replace('', pd.NA).dropna()

<ipython-input-5-4012012bd243>:41: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Liens'] = df2['Liens'].str.replace('./data', '/content/ALFFA_PUBLIC/ASR/FONGBE/data').str.strip()


In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [7]:
fongbe = DatasetDict({'train': Dataset.from_pandas(df_train), 'test': Dataset.from_pandas(df_test)})

In [8]:
fongbe

DatasetDict({
    train: Dataset({
        features: ['file', 'transcription', '__index_level_0__'],
        num_rows: 8234
    })
    test: Dataset({
        features: ['file', 'transcription', '__index_level_0__'],
        num_rows: 2168
    })
})

In [9]:
chars_to_ignore_regex = '[\"\?\.\!\-\;\:\(\)\,]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [10]:
fongbe = fongbe.map(remove_special_characters)

Map:   0%|          | 0/8234 [00:00<?, ? examples/s]

Map:   0%|          | 0/2168 [00:00<?, ? examples/s]

In [11]:
show_random_elements(fongbe['train'], num_examples=1)

,file,transcription,__index_level_0__
0,/content/ALFFA_PUBLIC/ASR/FONGBE/data/train/wav/denise/denise_fongbe_corp012_010.wav,e ɖo glagla ɖu wε,844


In [12]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocabs = fongbe.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fongbe.column_names["train"])

Map:   0%|          | 0/8234 [00:00<?, ? examples/s]

Map:   0%|          | 0/2168 [00:00<?, ? examples/s]

In [14]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))


In [15]:
vocab_list

[' ',
 'á',
 't',
 'ó',
 'g',
 'ɔ',
 'w',
 'ε',
 'é',
 'z',
 'è',
 '̀',
 'ὲ',
 'ɖ',
 'ò',
 '̆',
 'h',
 'v',
 's',
 'ù',
 'ì',
 'p',
 'ú',
 'ŏ',
 'ĕ',
 'o',
 'à',
 'i',
 'j',
 'r',
 'k',
 'n',
 'ĭ',
 'u',
 'm',
 'c',
 'a',
 '́',
 'x',
 'd',
 'b',
 'l',
 'e',
 'f',
 'έ',
 'í',
 'ɛ',
 'y',
 'ŭ']

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 'á': 1,
 't': 2,
 'ó': 3,
 'g': 4,
 'ɔ': 5,
 'w': 6,
 'ε': 7,
 'é': 8,
 'z': 9,
 'è': 10,
 '̀': 11,
 'ὲ': 12,
 'ɖ': 13,
 'ò': 14,
 '̆': 15,
 'h': 16,
 'v': 17,
 's': 18,
 'ù': 19,
 'ì': 20,
 'p': 21,
 'ú': 22,
 'ŏ': 23,
 'ĕ': 24,
 'o': 25,
 'à': 26,
 'i': 27,
 'j': 28,
 'r': 29,
 'k': 30,
 'n': 31,
 'ĭ': 32,
 'u': 33,
 'm': 34,
 'c': 35,
 'a': 36,
 '́': 37,
 'x': 38,
 'd': 39,
 'b': 40,
 'l': 41,
 'e': 42,
 'f': 43,
 'έ': 44,
 'í': 45,
 'ɛ': 46,
 'y': 47,
 'ŭ': 48}

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

51

In [19]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [20]:
tokenizer = Wav2Vec2CTCTokenizer("/content/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [21]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [22]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [23]:
#import soundfile as sf
import librosa
import warnings

warnings.filterwarnings("ignore")

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch['file'], sr = 16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [24]:
fongbe = fongbe.map(speech_file_to_array_fn, remove_columns=fongbe.column_names["train"], num_proc=1)

Map:   0%|          | 0/8234 [00:00<?, ? examples/s]

Map:   0%|          | 0/2168 [00:00<?, ? examples/s]

In [25]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [26]:
fongbe_prepared = fongbe.map(prepare_dataset, remove_columns=fongbe.column_names["train"], batch_size=32, batched=True)

Map:   0%|          | 0/8234 [00:00<?, ? examples/s]

Map:   0%|          | 0/2168 [00:00<?, ? examples/s]

In [27]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
!pip install  jiwer

In [30]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [31]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [32]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model.freeze_feature_extractor()

In [34]:
torch.cuda.is_available()

True

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/fongbe",
  group_by_length=True,
  per_device_train_batch_size=3,
  evaluation_strategy="steps",
  num_train_epochs=3,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=3,
  report_to='wandb'
)

In [41]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=fongbe_prepared["train"],
    eval_dataset=fongbe_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [42]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Wer
500,3.761800,2.765115,0.939803
1000,2.214600,1.059409,0.668223
1500,1.097500,0.793101,0.537450
2000,0.848600,0.705940,0.493340
2500,0.747000,0.722555,0.476042
3000,0.677300,0.669860,0.410223
3500,0.597600,0.696166,0.379519
4000,0.529900,0.697688,0.374935
4500,0.513900,0.743582,0.372427
5000,0.472800,0.638419,0.307905


TrainOutput(global_step=8235, training_loss=0.8344562403129387, metrics={'train_runtime': 3839.7151, 'train_samples_per_second': 6.433, 'train_steps_per_second': 2.145, 'total_flos': 1.9122140152668157e+18, 'train_loss': 0.8344562403129387, 'epoch': 3.0})

In [44]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [46]:
# Téléversez le modèle et le processeur sur Hugging Face
model.push_to_hub("billfass/fongbe_asr_model")
processor.push_to_hub("billfass/fongbe_asr_model")

CommitInfo(commit_url='https://huggingface.co/billfass/fongbe_asr_model/commit/e6b3e13048ba9cce377cad222832132fca2f2329', commit_message='Upload processor', commit_description='', oid='e6b3e13048ba9cce377cad222832132fca2f2329', pr_url=None, pr_revision=None, pr_num=None)